# Recommender with subset of item/user attributes

## Introduction

In this notebook, we develop a personalized recommender system for tech products using a subset of user and item attributes. Starting with preprocessed data—including user id,age  and product id and price—we construct a two-tower neural network model that learns to match users with relevant items.

The pipeline includes steps such as:

* Preparing positive and negative interaction samples

* Feature preprocessing (encoding and scaling)

* Building and training the recommendation model

* Generating and displaying recommendations for diverse user personalities

* Evaluating the model’s predictive performance using a holdout set

This approach allows us to build a recommender that not only predicts user preferences accurately but also adapts to varying user profiles and product characteristics.

## Installations and Imports

In [ ]:
# Install required packages
!pip install -q tensorflow scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, Model
from pprint import pprint
from sklearn.model_selection import train_test_split

## Data Preparation and Holdout Set Creation for Recommender Training

In this phase, we begin building the recommender system using a subset of user and item attributes. The user, item, and interaction data are first loaded from the previously generated CSV files: users.csv, items.csv, and user_product_rating_matrix.csv. These files were created during the earlier stage of data generation and represent our chosen domain — tech products.

The interaction matrix is reshaped into a user-item-label format, focusing solely on positive interactions (i.e., where a user has meaningfully interacted with a product). To train and evaluate the model, we randomly split the positive interactions into 80% training and 20% holdout sets.

The holdout set is crucial as it allows us to test the recommender's ability to predict items that a user has already interacted with — simulating how well the model can recover known preferences. This step ensures that our evaluation is grounded in realistic scenarios and supports the development of a more effective recommendation strategy.

In [ ]:
users = pd.read_csv('users.csv')


items = pd.read_csv('items.csv')
ratings_df = pd.read_csv('user_product_rating_matrix.csv')

# Create user-item pairs for training based on ratings_df
interactions_df = ratings_df.melt(id_vars=['user_id'], var_name='item_id', value_name='label')
interactions_positve=interactions_df[interactions_df['label'] == 1]

In [ ]:
from sklearn.model_selection import train_test_split

# Shuffle and split interactions (assuming all are positive)
interactions_shuffled = interactions_positve.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into 80% train, 20% holdout
interactions_train, interactions_holdout = train_test_split(
    interactions_shuffled,
    test_size=0.2,
    random_state=42
)

# Optional: reset indices
interactions = interactions_train.reset_index(drop=True)
interactions_holdout = interactions_holdout.reset_index(drop=True)

In [ ]:
print(users.shape)
print(items.shape)
print(interactions.shape)
print(interactions_holdout.shape)

(10000, 4)
(1000, 5)
(773028, 3)
(193258, 3)


## Negative Sampling for Balanced Interaction Modeling

To improve the robustness of our recommender system and train a model that can effectively distinguish between relevant and irrelevant items, we implement negative sampling.

In real-world scenarios, users interact with only a small fraction of available products — leaving most items unobserved. Treating all non-interactions as negative signals can lead to bias. To address this, we sample a manageable and representative set of negative examples.

Specifically, we:

* Extract all negative user-item pairs (label = 0).

* Shuffle these globally to ensure randomness.

* Limit the number of negatives per user by selecting the top 100 for each user.

* Combine these negatives with the previously extracted positive interactions (label = 1).

This balanced dataset of positive and negative interactions allows us to train models that better understand user preferences and make more accurate predictions.

In [ ]:
import pandas as pd
from tqdm import tqdm

# Step 1: Filter to only negative samples
negatives = interactions_df[interactions_df['label'] == 0]

# Step 2: Shuffle the negatives globally
negatives = negatives.sample(frac=1, random_state=42).reset_index(drop=True)

# Step 3: For each user, take the first 100 negatives
neg_samples = (
    negatives.groupby('user_id', group_keys=False)
    .head(100)
)

# Step 4: Mark original interactions (positives) with label = 1, if not already
interactions['label'] = 1

# Step 5: Combine positives and sampled negatives
enhanced_interactions = pd.concat([interactions, neg_samples], ignore_index=True)
interactions=enhanced_interactions


In [ ]:
# new shape with also 100 extra non interactions per user
interactions.shape

(1773028, 3)

## Preprocessing User and Item Attributes for Model Input


Before feeding the data into the recommender model, we preprocess user and item attributes to ensure they are in a format suitable for machine learning algorithms.

We begin by merging relevant user (e.g., age) and item (e.g., price) attributes into the interaction dataset. Then, we apply the following preprocessing steps:

* Label Encoding: We convert the categorical identifiers (user_id and item_id) into numerical representations using LabelEncoder. This transformation is essential for embedding layers in recommendation models.

* Feature Scaling: Numerical features like user age and product price are scaled using MinMaxScaler to normalize their values between 0 and 1. This prevents features with larger ranges from dominating the learning process.

These steps ensure that both categorical and numerical attributes are efficiently utilized by the model, leading to improved learning and better generalization.

In [ ]:
# Merge age and price
interactions = interactions.merge(users, on='user_id')
interactions = interactions.merge(items, on='item_id')

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

interactions['user_encoded'] = user_encoder.fit_transform(interactions['user_id'])
interactions['item_encoded'] = item_encoder.fit_transform(interactions['item_id'])

# Scale numerical features
scaler_age = MinMaxScaler()
scaler_price = MinMaxScaler()

interactions['age_scaled'] = scaler_age.fit_transform(interactions[['age']])
interactions['price_scaled'] = scaler_price.fit_transform(interactions[['price']])


## Building a Two-Tower Neural Network Recommender


In this section, we implement a two-tower neural network recommender that learns separate representations for users and items before combining them to predict interactions.

The model is structured as follows:

* User Tower: Processes user-specific inputs — a unique user ID and their scaled age. The user ID is passed through an embedding layer, flattened, and then concatenated with the age input. This merged vector is then transformed via a dense layer.

* Item Tower: Mirrors the user tower but operates on item-specific inputs — item ID and scaled price. The embedded item ID and item price are similarly combined and passed through a dense layer.

* Interaction Layer: The outputs of both towers (user and item representations) are combined using a dot product, which captures their similarity. A sigmoid activation then predicts the probability of interaction (i.e., whether the user would interact with the item).

We compile the model using binary cross-entropy loss and the Adam optimizer, training it over 5 epochs with a batch size of 64.

This two-tower architecture enables efficient learning of user-item relationships and can scale to large datasets, making it a strong baseline for personalized recommendation.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

embedding_dim = 32

# Inputs
user_id_input = Input(shape=(), name='user_id', dtype=tf.int32)
user_age_input = Input(shape=(1,), name='user_age', dtype=tf.float32)

item_id_input = Input(shape=(), name='item_id', dtype=tf.int32)
item_price_input = Input(shape=(1,), name='item_price', dtype=tf.float32)

# User tower
user_embedding = layers.Embedding(len(user_encoder.classes_), embedding_dim)(user_id_input)
user_embedding = layers.Flatten()(user_embedding)
user_concat = layers.Concatenate()([user_embedding, user_age_input])
user_vector = layers.Dense(64, activation='relu')(user_concat)

# Item tower
item_embedding = layers.Embedding(len(item_encoder.classes_), embedding_dim)(item_id_input)
item_embedding = layers.Flatten()(item_embedding)
item_concat = layers.Concatenate()([item_embedding, item_price_input])
item_vector = layers.Dense(64, activation='relu')(item_concat)

# Dot product
dot_product = layers.Dot(axes=1)([user_vector, item_vector])
output = layers.Activation('sigmoid')(dot_product)

# Compile
model = Model(inputs=[user_id_input, user_age_input, item_id_input, item_price_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Build input
X = {
    'user_id': interactions['user_encoded'].values,
    'user_age': interactions['age_scaled'].values,
    'item_id': interactions['item_encoded'].values,
    'item_price': interactions['price_scaled'].values
}
y = interactions['label'].values

model.fit(X, y, epochs=5, batch_size=64)


Epoch 1/5
27704/27704 ━━━━━━━━━━━━━━━━━━━━ 80s 3ms/step - accuracy: 0.6810 - loss: 0.6121
Epoch 2/5
27704/27704 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 0.7711 - loss: 0.5566
Epoch 3/5
27704/27704 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.7723 - loss: 0.5554
Epoch 4/5
27704/27704 ━━━━━━━━━━━━━━━━━━━━ 77s 3ms/step - accuracy: 0.7724 - loss: 0.5554
Epoch 5/5
27704/27704 ━━━━━━━━━━━━━━━━━━━━ 83s 3ms/step - accuracy: 0.7732 - loss: 0.5555


## Personalized Recommendation For Users

To enable personalized suggestions, we implemented a function that provides the top-N most relevant product recommendations for a specific user, based on the trained two-tower model.

The function follows these steps:

1. User Feature Extraction: Retrieves and encodes the user's ID and scales their age using the fitted scaler.

2. Candidate Item Pool: Considers all available items, excluding those the user has already interacted with positively.

3. Input Preparation: Constructs model inputs by replicating the user features for all remaining candidate items.

4. Prediction: The model computes interaction scores (likelihood of preference) for each user-item pair.

5. Ranking: The top-N items with the highest scores are selected as recommendations.

6. Result Formatting: For each recommended item, we provide relevant metadata (e.g., description, price, type, and features), along with the predicted score.

This function supports flexible top-N queries and enables us to evaluate or demonstrate the recommender’s output for individual users, highlighting its personalization capabilities

In [ ]:
def recommend_for_user(user_id, top_n=5):
    # Get user encoded index and scaled age
    user_idx = user_encoder.transform([user_id])[0]
    user_row = users[users['user_id'] == user_id].iloc[0]
    user_age_scaled = scaler_age.transform([[user_row['age']]])[0][0]

    # All item data
    all_item_ids = items['item_id'].values
    all_item_indices = item_encoder.transform(all_item_ids)
    item_prices = items['price'].values
    item_prices_scaled = scaler_price.transform(item_prices.reshape(-1, 1)).flatten()

    # Remove already interacted items
    interacted_items = interactions[
        (interactions['user_id'] == user_id) & (interactions['label'] == 1)
    ]['item_id'].values
    interacted_indices = item_encoder.transform(interacted_items)

    mask = ~np.isin(all_item_indices, interacted_indices)
    item_indices = all_item_indices[mask]
    item_ids = all_item_ids[mask]
    item_prices_scaled = item_prices_scaled[mask]

    # Prepare model input
    user_id_array = np.full_like(item_indices, user_idx)
    user_age_array = np.full_like(item_prices_scaled, user_age_scaled)

    # Predict scores
    predictions = model.predict({
        'user_id': user_id_array,
        'user_age': user_age_array,
        'item_id': item_indices,
        'item_price': item_prices_scaled
    }, verbose=0).flatten()

    # Select top N
    top_idxs = predictions.argsort()[-top_n:][::-1]
    top_item_ids = item_ids[top_idxs]
    top_scores = predictions[top_idxs]

    # Merge and format output
    recommended = []
    for item_id, score in zip(top_item_ids, top_scores):
        item_row = items[items['item_id'] == item_id].iloc[0]
        rec = {
            'user_id': user_row['user_id'],
            'user_age': user_row['age'],
            'user_profession': user_row.get('profession', 'N/A'),
            'user_personality': user_row.get('personality', 'N/A'),
            'item_id': item_row['item_id'],
            'item_type': item_row.get('product_type', 'N/A'),
            'item_description': item_row.get('description', 'N/A'),
            'item_price': item_row['price'],
            'item_features': item_row.get('features', 'N/A'),
            'score': float(score)
        }
        recommended.append(rec)

    return recommended


In [ ]:
# print recommendations for a random user
pprint(recommend_for_user('bud_0002', top_n=3))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[{'item_description': 'With noise cancellation and wireless connectivity, this '
                      'headphones is great for travel. Perfect for music '
                      'lovers, it delivers top-tier functionality.',
  'item_features': 'Fast charging, Bluetooth 5.0',
  'item_id': 'item_0755',
  'item_price': np.float64(43.5),
  'item_type': 'headphones',
  'score': 0.9996224641799927,
  'user_age': np.int64(26),
  'user_id': 'bud_0002',
  'user_personality': 'Budget Conscious',
  'user_profession': 'education'},
 {'item_description': 'With wireless connectivity and noise cancellation, this '
                      'headphones is great for travel. Perfect for music '
                      'lovers, it delivers top-tier functionality.',
  'item_features': 'Bluetooth 5.0, Over-ear design, Noise cancellation, Fast '
                   'charging',
  'item_id': 'item_0341',
  'item_price': np.float64(46.06),
  'item_type': 'headphones',
  'score': 0.9976105690002441,
  'user_age': np.i

## Exploring Recommendations Across Personality Types

To better understand how the recommender adapts to different user profiles, we generated indicative recommendations for a sample of users from each personality type.

For each personality group, we randomly selected up to three users, and for each user, we retrieved the top-5 personalized recommendations using our trained model. These recommendations were then printed along with relevant user and item details, offering insights into:

* The diversity and relevance of recommended tech products.

* How user attributes, such as age and personality, influence the system’s output.

* Potential patterns or differences in recommendations across personality types.

This qualitative step serves as an initial diagnostic of model behavior, helping us assess whether personalization is occurring meaningfully across segments before moving on to quantitative evaluation.

In [ ]:
import random
from pprint import pprint

def recommend_for_representative_users(users_df, top_n=5, users_per_personality=3):
    personalities = users_df['personality'].unique()

    for personality in personalities:
        print(f"\n Personality: {personality}")
        personality_users = users_df[users_df['personality'] == personality]['user_id'].tolist()

        if len(personality_users) < users_per_personality:
            selected_users = personality_users
        else:
            selected_users = random.sample(personality_users, users_per_personality)

        for user_id in selected_users:
            print(f"\n Recommendations for user: {user_id}")
            recs = recommend_for_user(user_id, top_n=top_n)
            pprint(recs)


In [ ]:
# print recommedations for 3 users per personality
recommend_for_representative_users(users, top_n=5, users_per_personality=3)


 Personality: Tech Enthusiast

 Recommendations for user: tec_0348


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[{'item_description': 'This laptop features advanced cooling system and '
                      'AI-powered processor, designed for gamers. It is built '
                      'for performance, offering reliable performance and user '
                      'comfort.',
  'item_features': 'Fast charging, Bluetooth 5.0',
  'item_id': 'item_0849',
  'item_price': np.float64(483.54),
  'item_type': 'laptop',
  'score': 0.9993810653686523,
  'user_age': np.int64(35),
  'user_id': 'tec_0348',
  'user_personality': 'Tech Enthusiast',
  'user_profession': 'engineering'},
 {'item_description': 'Laptop comes equipped with advanced cooling system and '
                      'high refresh rate display, making it built for '
                      'performance. Great choice for business professionals.',
  'item_features': 'Touchscreen, Expandable storage, Fast charging',
  'item_id': 'item_0768',
  'item_price': np.float64(1612.27),
  'item_type': 'laptop',
  'score': 0.9993674159049988,
  'user_age'

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[{'item_description': 'Laptop comes equipped with long battery life and '
                      'advanced cooling system, making it built for '
                      'performance. Great choice for business professionals.',
  'item_features': 'Fast charging, Expandable storage',
  'item_id': 'item_0732',
  'item_price': np.float64(407.81),
  'item_type': 'laptop',
  'score': 0.9997192025184631,
  'user_age': np.int64(30),
  'user_id': 'tec_1161',
  'user_personality': 'Tech Enthusiast',
  'user_profession': 'engineering'},
 {'item_description': 'Laptop comes equipped with advanced cooling system and '
                      'long battery life, making it built for performance. '
                      'Great choice for remote workers.',
  'item_features': 'USB-C connectivity, Bluetooth 5.0, Touchscreen',
  'item_id': 'item_0631',
  'item_price': np.float64(285.36),
  'item_type': 'laptop',
  'score': 0.9996629953384399,
  'user_age': np.int64(30),
  'user_id': 'tec_1161',
  'user_personali

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[{'item_description': 'Designed for gamers, this headphones includes wireless '
                      'connectivity and noise cancellation. It’s great for '
                      'travel and built for high efficiency.',
  'item_features': 'Noise cancellation, Fast charging, Bluetooth 5.0',
  'item_id': 'item_0967',
  'item_price': np.float64(49.19),
  'item_type': 'headphones',
  'score': 0.9983124732971191,
  'user_age': np.int64(26),
  'user_id': 'bud_1133',
  'user_personality': 'Budget Conscious',
  'user_profession': 'arts'},
 {'item_description': 'Headphones comes equipped with deep bass and noise '
                      'cancellation, making it great for travel. Great choice '
                      'for commuters.',
  'item_features': 'Bluetooth 5.0, Fast charging, Noise cancellation, Over-ear '
                   'design',
  'item_id': 'item_0569',
  'item_price': np.float64(40.0),
  'item_type': 'headphones',
  'score': 0.9982671737670898,
  'user_age': np.int64(26),
  'user_i

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[{'item_description': 'With ergonomic design and adjustable DPI, this mouse is '
                      'ideal for everyday use. Perfect for gamers, it delivers '
                      'top-tier functionality.',
  'item_features': 'Silent clicks, Rechargeable, Compact design, Wireless',
  'item_id': 'item_0380',
  'item_price': np.float64(31.07),
  'item_type': 'mouse',
  'score': 0.9966893196105957,
  'user_age': np.int64(24),
  'user_id': 'bud_1631',
  'user_personality': 'Budget Conscious',
  'user_profession': 'education'},
 {'item_description': 'Laptop comes equipped with long battery life and '
                      'advanced cooling system, making it great for travel. '
                      'Great choice for business professionals.',
  'item_features': 'Fast charging, USB-C connectivity',
  'item_id': 'item_0197',
  'item_price': np.float64(427.55),
  'item_type': 'laptop',
  'score': 0.9957559704780579,
  'user_age': np.int64(24),
  'user_id': 'bud_1631',
  'user_personality': 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


## Evaluating Recommender Accuracy with Holdout Interactions


To assess the performance of our recommender, we evaluated its ability to predict real, previously observed interactions using the holdout dataset. This dataset was reserved during training and contains genuine user-item interactions that the model has not seen before, making it ideal for unbiased evaluation.

For each user in the holdout set, we randomly selected one actual interaction and calculated the model's predicted score for that user-item pair using the trained two-tower architecture.

The resulting average predicted score for these true interactions was:

68.80%

This score indicates the model's confidence in recommending items that users have genuinely engaged with, providing a quantitative measure of accuracy and validating that the model has learned meaningful patterns during training.

In [ ]:
def get_score(user_id, item_id):
    # Encode user and item
    user_idx = user_encoder.transform([user_id])[0]
    item_idx = item_encoder.transform([item_id])[0]

    # Get user and item features
    user_row = users[users['user_id'] == user_id].iloc[0]
    item_row = items[items['item_id'] == item_id].iloc[0]

    user_age_scaled = scaler_age.transform([[user_row['age']]])[0][0]
    item_price_scaled = scaler_price.transform([[item_row['price']]])[0][0]

    # Predict
    prediction = model.predict({
        'user_id': np.array([user_idx]),
        'user_age': np.array([user_age_scaled]),
        'item_id': np.array([item_idx]),
        'item_price': np.array([item_price_scaled])
    }, verbose=0).flatten()[0]

    return float(prediction)


In [ ]:
score = get_score('bud_0002', 'item_0755')
print(f"Predicted score: {score:.4f}")


Predicted score: 0.9996


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:

interactions[(interactions['user_id']=='bud_0002') & (interactions['item_id']=='item_0755')]

,user_id,item_id,label,age,profession,personality,product_type,description,price,features,user_encoded,item_encoded,age_scaled,price_scaled


In [ ]:

interactions_holdout[(interactions_holdout['user_id']=='bud_0002') & (interactions_holdout['item_id']=='item_0755')]

,user_id,item_id,label
117458,bud_0002,item_0755,1


In [ ]:
from tqdm import tqdm

def evaluate_holdout_one_per_user(holdout_df):
    # Pick one random interaction per user
    sampled = holdout_df.groupby('user_id').sample(n=1, random_state=42).reset_index(drop=True)

    scores = []
    for _, row in tqdm(sampled.iterrows(), total=len(sampled), desc="Scoring one interaction per user"):
        user_id = row['user_id']
        item_id = row['item_id']
        score = get_score(user_id, item_id)
        scores.append(score)

    avg_score = sum(scores) / len(scores)
    #print(f"\n Average predicted score (1 interaction per user): {avg_score:.4f}")
    return avg_score



In [ ]:
avg_one_per_user = evaluate_holdout_one_per_user(interactions_holdout)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
Scoring one interaction per user:  88%|████████▊ | 8751/10000 [21:51<03:09,  6.60it/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
Scoring one interaction per user:  88%|████████▊ | 8752/10000 [21:51<03:03,  6.80it/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11

In [ ]:
print(f"\n Average recommendation score for actual interactions "
      f"(1 random real interaction per user from holdout): {(avg_one_per_user * 100):.4f}%")


 Average recommendation score for actual interactions (1 random real interaction per user from holdout): 68.7972%
